In [1]:
import socket
import pickle
import struct
import cv2
import numpy as np
import os
import torch
import time
from ultralytics import YOLO
from tensorflow.keras.models import load_model

model = YOLO('yolov8s.pt', 'YOLO8.yaml')

# 저장된 가중치 파일 로드
#loaded_weights = torch.load('YOLO8_trained_model1118.pt')
loaded_weights = torch.load('YOLO8_trained_weights0517.pt')
# 모델에 state_dict 로드
model.load_state_dict(loaded_weights)

os.environ["CUDA_VISIBLE_DEVICES"]="0"  

def send_data(data):
    client_socket.send(data.encode()) # 기본자세로 다시 옮김
    time.sleep(0.01) # 0.01초 대기

In [ ]:
#2024/11/20
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host_ip = '서버 주소'     # 서버 IP 주소
port = 1111                  # 포트
client_socket.connect((host_ip, port))

data = b""                # 데이터를 바이트 형식으로 받기
payload_size = struct.calcsize("Q")
class_names = {0: 'tom_flower', 1: 'truss', 2: 'stem'}    # 3개의 클래스 0 : 만개꽃, 1 : 화방, 2 : 줄기


# 미리 초기화를 해야하는 변수들
phase_num = 0      # 페이즈 제어 변수
control_num = 0    # 종합정인 제어 변수
truss_control = 0   # 화방 개수 변수
truss_coordinate = 0  # 화방 개수에 따른 위치 조정 변수
flower_control = 0   # 만개꽃 페이즈 개수 변수
noflower_count = 0   # 만개꽃 개수 카운트 변수, 2개이상 충족이 안될경우 다른 쪽으로 넘김
final_truss_num = 0
truss_num_is_0 = 0
truss_num_is_1 = 0
truss_num_is_2 = 0
final_up_down = 0
false_truss_detect = 0

while True:
    # 프레임 수신
    while len(data) < payload_size:
        packet = client_socket.recv(4*1024)  # 4K
        if not packet: break
        data += packet
    if not data: break
    packed_msg_size = data[:payload_size]
    data = data[payload_size:]
    msg_size = struct.unpack("Q", packed_msg_size)[0]
    
    while len(data) < msg_size:
        data += client_socket.recv(4*1024)

    frame_data = data[:msg_size]
    data = data[msg_size:]
    frame = pickle.loads(frame_data)
    frame = cv2.resize(frame, (640, 480))
    result = model(frame, conf=0.3, iou=0.5)

    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = np.transpose(img, (2, 0, 1))
    img_tensor = torch.from_numpy(img).unsqueeze(0).float()
    img_tensor /= 255.0

    with torch.no_grad():
        results = model(img_tensor)
    
    # 바운딩 박스 컬러 설정
    class_colors = {'tom_flower': (255, 0, 0), 'truss': (0, 255, 0), 'stem': (0, 0, 255) }
    
    # 감지하면 찾을 객체들
    detected_objects = {'boxes': [], 'classes': [], 'scores': [] }

        
    for i, result in enumerate(results):
        
        # 프레임 처리 시작 시 객체 정보들 초기화, 프레임마다 객체를 초기화 해서 찾아야 하기 때문에 진행
        tom_flower_num = 0  #만개꽃 개수 증가
        truss_num = 0

        tom_stem = []       # 탐지마다 갱신되는 줄기 
        tom_truss = []      # 탐지마다 갱신되는 화방
        tom_flowers1 = []   # 1화방 중심과 가장 가까운 만개꽃들
        tom_flowers2 = []   # 2화방 중심과 가장 가까운 만개꽃들

        # NMS를 위한 바운딩 박스와 신뢰도 저장
        boxes = []
        scores = []
        classes = []

        for box, cls in zip(result.boxes.xyxy, result.boxes.cls):
            x1, y1, x2, y2 = box.cpu().numpy()
            class_index = int(cls.cpu())
            score = result.boxes.conf[i].cpu().numpy().item()

            boxes.append([x1, y1, x2, y2])
            scores.append(score)
            classes.append(class_index)

        # NMS 적용
        indices = cv2.dnn.NMSBoxes(boxes, scores, score_threshold=0.3, nms_threshold=0.4)

        for index in indices:
            index = index[0]  # index는 2D 배열로 반환되므로 첫 번째 요소를 가져옴
            box = boxes[index]
            class_index = classes[index]
            score = scores[index]

            # 바운딩 박스 좌표
            x1, y1, x2, y2 = box

            class_name = class_names.get(class_index, 'Unknown')
            color = class_colors.get(class_name, (255, 255, 255))

            # 바운딩 박스 그리기
            start_point = (int(x1), int(y1))
            end_point = (int(x2), int(y2))
            thickness = 2
            cv2.rectangle(frame, start_point, end_point, color, thickness)

            # 클래스 이름과 신뢰도 표시
            label = f'{class_name}: {score:.2f}'
            cv2.putText(frame, label, (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # 객체 정보를 리스트에 추가
            detected_objects['boxes'].append((x1, y1, x2, y2))
            detected_objects['classes'].append(class_name)
            detected_objects['scores'].append(score)

            
            if class_index == 2:    # 줄기 처리
                stem_info = {
                    'y_center': y_center,
                    'x_center': x_center,
                    'box': (x1, y1, x2, y2),
                    'class_index': class_index,
                }
                tom_stem.append(stem_info)                 # tom_stem 리스트에 추가
                tom_stem.sort(key=lambda x: x['y_center'], reverse=True) # tom_stem 리스트 y좌표 낮은 순으로 정렬
            
            
            if class_index == 1:    # 화방 처리
                truss_info = {
                    'y_center': y_center,
                    'x_center': x_center,
                    'box': (x1, y1, x2, y2),
                    'class_index': class_index,
                }
                tom_truss.append(truss_info)                # tom_truss 리스트에 추가
                tom_truss.sort(key=lambda x: x['y_center'], reverse=True) # tom_truss 리스트 y좌표 낮은 순으로 정렬
                truss_num += 1                              # 화방당 개수 증가
                

            if class_index == 0:    # 만개꽃 처리
                flower_info = {
                    'y_center': y_center,
                    'x_center': x_center,
                    'box': (x1, y1, x2, y2),
                    'class_index': class_index,
                }
                tom_flowers1.append(flower_info)  # tom_flowers1 리스트에 추가
                tom_flowers2.append(flower_info)  # tom_flowers2 리스트에 추가
                tom_flower_num += 1               # 만개꽃 개수 증가

                # 이번 프레임에서 만개꽃이 2개이상 검출되었고, 화방이 있으며 1화방과 가까운 순으로 정렬 
                flower_in_truss1 = []
                if len(tom_truss) > 0 and len(tom_flowers1) > 0:
                    truss1_box = tom_truss[0]['box']  # tom_truss[0]의 바운딩 박스
                    for flower in tom_flowers1:
                        # 만개꽃의 중심좌표가 tom_truss[0]의 바운딩 박스 내에 있는지 확인
                        if truss1_box[0] <= flower['x_center'] <= truss1_box[2] and truss1_box[1] <= flower['y_center'] <= truss1_box[3]:
                            flower_in_truss1.append(flower)  # 조건을 만족하는 만개꽃을 flower_in_truss1에 추가
                
                # tom_truss[1]의 바운딩 박스 내에 있는 tom_flowers2의 만개꽃 중심좌표를 기준으로 필터링
                flower_in_truss2 = []
                if len(tom_truss) > 1 and len(tom_flowers2) > 0:
                    truss2_box = tom_truss[1]['box']  # tom_truss[1]의 바운딩 박스
                    for flower in tom_flowers2:
                        # 만개꽃의 중심좌표가 tom_truss[1]의 바운딩 박스 내에 있는지 확인
                        if truss2_box[0] <= flower['x_center'] <= truss2_box[2] and truss2_box[1] <= flower['y_center'] <= truss2_box[3]:
                            flower_in_truss2.append(flower)  # 조건을 만족하는 만개꽃을 flower_in_truss2에 추가


#======================================== 시스템시작, phase_num == 0은 주행 페이즈 =============================================================    
        
        if(phase_num == 0):     
            print("phase_num = ", phase_num)
            if len(tom_stem) > 0 and tom_stem[0] is not None and (tom_stem[0]['x_center']>=250 and tom_stem[0]['x_center']<=350):
                control_num += 1
                if(control_num > 2):  # 프레임 내 줄기가 중앙에 완전히 위치했다 판단하는 경우, 페이즈 넘버 1로 전환
                    send_data("1")    # 데이터 1 전송
                    phase_num = 1
                    control_num=0

#========================================  화방 정보 분석 페이즈 ==============================================================================

        elif(phase_num == 1):    
            control_num += 1
            # 프레임당 검출된 화방개수를 판단하기 위해 검출된 화방 수에 따라서 truss_is_num_n에 추가되는 수가 달라짐
            if(truss_num==0):         # 화방 개수 0
                truss_num_is_0 += 1   
            elif(truss_num==1):       # 화방 개수 1
                truss_num_is_1 += 1
            elif(truss_num>=2):       # 화방 개수 2개 이상
                truss_num_is_2 += 1

            # 프레임마다 화방이 몇개 검출되었는지 확인하는 코드 
            print("phase_num = ", phase_num)
            print("truss_num_is_0 = ", truss_num_is_0)  
            print("truss_num_is_1 = ", truss_num_is_1)
            print("truss_num_is_2 = ", truss_num_is_2)
            print("control_num = ", control_num)


            # 화방개수가 2개를 충족하지 못하는 경우 팔을 올려 화방을 확인한다.
            if(control_num%30 == 29):
                print("2화방을 모두 찾을때까지 팔을 계속 올립니다. ", control_num%29)
                send_data("4")    # 데이터 9 전송
                print("UP")


            if(truss_num_is_2 >= 15):
                print("최종 화방은 2개")
                final_truss_num = 2     # 이번 사이클은 화방이 2개라 생각하고 진행
                flower_control = 1      # flower_control = 0이면 만개꽃 개수 판단때, 1화방만 있다고 생각함
                send_data("1")          # 정자세로 돌리기위해 데이터 1 전송
                phase_num = 3           # 화방 위치 조정으로 들어감
                control_num = 0

            if(truss_num_is_2 < 15 and truss_num_is_1 >= 280):
                print("최종 화방은 1개")
                final_truss_num = 1     # 이번 사이클은 화방이 2개라 생각하고 진행
                flower_control = 0      # flower_control = 0이면 만개꽃 개수 판단때, 1화방만 있다고 생각함
                send_data("1")          # 정자세로 돌리기위해 데이터 1 전송
                phase_num = 3           # 화방 위치 조정으로 들어감
                control_num = 0

            if(truss_num_is_0 > 300):
                print("최종 화방은 0개")
                final_truss_num = 0     # 이번 사이클은 화방이 0개라 생각하고 진행
                send_data("1")          # 정자세로 돌리기위해 데이터 1 전송
                phase_num = 10          # 페이즈 10으로 돌아감
                control_num = 0

        
#====================================================== 화방 좌우, 높이 조정 페이즈 ============================================================
        
        elif(phase_num == 3):    
            control_num += 1
            print("truss_coordinate = ", truss_coordinate)
            print("phase_num = ", phase_num)
            print("len(tom_truss) = ",len(tom_truss))
            
            # 1화방을 찾지 못한 경우 false_truss_detect을 증가하고, 이걸 5의 배수마다 확인하면 UP을 해 로봇팔을 위로 올림
            if(truss_coordinate == 0 and len(tom_truss) <= 0):
                false_truss_detect += 1
                if (false_truss_detect < 90 and false_truss_detect%10 == 9):
                    send_data("4")    # 데이터 4 전송
                    print("1 화방을 찾지 못해 팔을 올립니다. send data = 4, UP")

                if(false_truss_detect > 100):
                    send_data("1")    # 정자세로 돌아가고 페이즈 10으로 귀환
                    print("1 화방을 감지하지 못해 페이즈 10으로 돌아갑니다. ")
                    false_truss_detect = 0
                    phase_num = 10
                    control_num = 0
                    
            # 2화방을 찾지 못한 경우 false_truss_detect을 증가하고, 이걸 5의 배수마다 확인하면 UP을 해 로봇팔을 위로 올림
            if(truss_coordinate == 1 and len(tom_truss) <= 1):
                false_truss_detect += 1
                if (false_truss_detect < 90 and false_truss_detect%10 == 9):
                    send_data("4")    # 데이터 9 전송
                    print("2화방을 찾지 못해 팔을 올립니다. send data = 4, UP")

                if(false_truss_detect > 100):
                    send_data("1")    # 정자세로 돌아가고 페이즈 10으로 귀환
                    print("2 화방을 감지하지 못해 페이즈 10으로 돌아갑니다. ")
                    false_truss_detect = 0
                    phase_num = 10
                    control_num = 0
                    
            
            # truss_coordinate == 0이면, 1화방을 제어한다.
            if(truss_coordinate == 0 and (control_num%3==1)):
                if len(tom_truss) > 0 and tom_truss[0] is not None: # 1화방 제어
                    
                    if (tom_truss[0]['x_center'] < 280):            # 화방이 중앙보다 왼쪽에 있는 경우
                        send_data("2")                              # 로봇팔을 왼쪽으로 움직임
                        print("LEFT")
                    if (tom_truss[0]['x_center'] > 320):            # 화방이 중앙보다 오른쪽에 있는 경우
                        send_data("3")                              # 로봇팔을 오른쪽으로 움직임
                        print("RIGHT")

                    if (tom_truss[0]['y_center'] < 220):            # 화방이 중앙보다 위에 있는 경우
                        send_data("4")                              # 로봇팔을 위로 움직임 
                        print("UP")
                    if (tom_truss[0]['y_center'] > 260):            # 화방이 중앙보다 아래에 있는 경우
                        send_data("5")                              # 로봇팔을 아래로 움직임
                        print("DOWN")

                    # 화방 위치 조정 완료 조건 : tom_truss가 1개이상이고, 가장 낮은 화방이 비어있지 않고, x,y중앙값에 위치한경우
                    if((tom_truss[0]['x_center'] >= 280 and tom_truss[0]['x_center'] <= 320) and 
                       (tom_truss[0]['y_center'] >= 220 and tom_truss[0]['y_center'] <= 260)): 
                        
                        send_data("6")    # 중앙 값 만족으로 대기
                        print("center")
                        control_num += 2  # 중앙 값 만족하면 카운트 시작
                        
                    if(control_num > 120):  # 화방 중앙에 위치시 만개꽃 조정 페이즈로 넘어감
                        print("4단계 페이즈로 넘어감")
                        false_truss_detect = 0
                        control_num = 0
                        phase_num = 4

                        
            # truss_coordinate == 1이면, 2화방을 제어한다.
            if(truss_coordinate == 1 and (control_num%3==1)):
                if len(tom_truss) > 1 and tom_truss[1] is not None: # 1화방 제어
                    
                    if (tom_truss[1]['x_center'] < 280):            # 화방이 중앙보다 왼쪽에 있는 경우
                        send_data("2")                              # 로봇팔을 왼쪽으로 움직임
                        print("LEFT")
                    if (tom_truss[1]['x_center'] > 320):            # 화방이 중앙보다 오른쪽에 있는 경우
                        send_data("3")                              # 로봇팔을 오른쪽으로 움직임
                        print("RIGHT")

                    if (tom_truss[1]['y_center'] < 220):            # 화방이 중앙보다 위에 있는 경우
                        send_data("4")                              # 로봇팔을 위로 움직임 
                        print("UP")
                    if (tom_truss[1]['y_center'] > 260):            # 화방이 중앙보다 아래에 있는 경우
                        send_data("5")                              # 로봇팔을 아래로 움직임
                        print("DOWN")

                    # 화방 위치 조정 완료 조건 : tom_truss가 1개이상이고, 가장 낮은 화방이 비어있지 않고, x,y중앙값에 위치한경우
                    if((tom_truss[1]['x_center'] >= 280 and tom_truss[1]['x_center'] <= 320) and 
                       (tom_truss[1]['y_center'] >= 220 and tom_truss[1]['y_center'] <= 260)): 
                        
                        send_data("6")    # 중앙 값 만족으로 대기
                        print("center")
                        control_num += 2  # 중앙 값 만족하면 카운트 시작
                        
                    if(control_num > 120):  # 화방 중앙에 위치시 만개꽃 조정 페이즈로 넘어감
                        print("4단계 페이즈로 넘어감")
                        false_truss_detect = 0
                        control_num = 0
                        phase_num = 4
                        


#=================================================== 만개꽃 개수 세기 및 조정 페이즈 ===========================================================

        # flower_control == 0   
        # flower_control == 1 
        # flower_control ==  2 
        

        elif(phase_num == 4):    
            print("phase_num = ", phase_num)
            print("flower_control = ",flower_control)
            print("control_num = ", control_num)
            print("noflower_count = ", noflower_count)
            print("len(flower_in_truss1) = ",len(flower_in_truss1))
            print("len(flower_in_truss2) = ",len(flower_in_truss2))
            
            # 화방 1개, 따라서 1화방만 감지하고 그에따라 판단하면됨
            # 화방 1개 안에 만개꽃이 2개있는가?를 감지한다. 화방 바운딩 박스내에 만개꽃 중심 좌표가 있는지를 판별하도록 한다.
            if(flower_control == 0):
                if len(flower_in_truss1) >= 2 :
                    control_num += 1
                    if(control_num<5) : 
                        print("대기")
                        send_data("6")
                    else:     # 만개꽃 개수 충족
                        print("1화방에 만개꽃 개수 충족, 분사페이즈로 넘어갑니다.")
                        control_num = 0
                        truss_control = 1  # 1화방만 있고, 1화방 분사해야함
                        phase_num = 5      # 분사 페이즈
                        noflower_count = 0    # 만개꽃 변수 초기화
                else : 
                    noflower_count += 1
                    if(noflower_count>30):
                        print("1화방에 만개꽃 개수 불충족, 경로주행으로 돌아갑니다.")
                        noflower_count = 0    # 만개꽃 변수 초기화
                        control_num = 0       # 컨트롤 변수 초기화 
                        noflower_count = 0
                        phase_num = 10        # 경로 주행 페이즈
                        
                        
            # 화방 2개, 1화방 조정
            elif(flower_control == 1):
                if len(flower_in_truss1) >= 2 :
                    control_num += 1
                    if(control_num<5) : 
                        send_data("6")
                    else:      # 만개꽃 위치 중앙으로 조정 완료
                        print("1화방에 만개꽃 개수 충족, 분사페이즈로 넘어갑니다.")
                        control_num = 0
                        truss_control = 2    # 2화방까지 있고, 1화방 분사해야함
                        flower_control = 2   # 다음 화방 조정은 2화방 조정으로 확인
                        truss_coordinate = 1 # 2화방 위치 조정을 하도록 설정
                        noflower_count = 0    # 만개꽃 변수 초기화
                        phase_num = 5        # 분사 페이즈
                else : 
                    noflower_count += 1
                    if(noflower_count>30):
                        print("1화방에 만개꽃 개수 불충족, 2화방 조정으로 넘어갑니다.")
                        truss_coordinate = 1
                        phase_num = 3         
                        flower_control = 2
                        control_num = 0
                        noflower_count = 0
                        truss_control = 1 # 2개의 화방중, 1화방 만개꽃 불충족, 2화방 탐지
                        
            # 화방 2개, 2화방 조절
            elif(flower_control == 2):
                if len(flower_in_truss2) >= 2 :
                    control_num += 1
                    if(control_num<10) : 
                        send_data("6")
                    else:  # 만개꽃 위치 중앙으로 조정 완료
                        print("2화방에 만개꽃 개수 충족, 분사페이즈로 넘어갑니다.")
                        phase_num = 5      # 분사 페이즈
                        truss_control = 1
                        control_num = 0
                        noflower_count = 0    # 만개꽃 변수 초기화
                        flower_control = 0
                else : 
                    noflower_count += 1
                    if(noflower_count>30):
                        print("2화방에 만개꽃 개수 불충족, 경로주행으로 돌아갑니다")
                        truss_coordinate = 0
                        phase_num = 10
                        flower_control = 0
                        noflower_count = 0    # 만개꽃 변수 초기화
                        control_num = 0

##=========================================================== 분사 페이즈 ===================================================================
        
        elif(phase_num == 5):   
            print("phase_num = ", phase_num)
            print("truss_control = ", truss_control)
            
            control_num += 1
            send_data("6")
            if(control_num == 5):
                send_data("7")    # 분사 
                print("분사")
            if(control_num> 20):
                if(truss_control == 1):    # truss_control == 1이면,  뭐가 되었든 화방 분사가 끝남
                    print("페이즈 10으로 넘어갑니다")
                    control_num = 0
                    phase_num = 10 
                    
                elif(truss_control == 2):  # truss_control == 2이면, 2화방이 중 1화방만 분사한거임
                    print("페이즈 3으로 돌아가 2화방 조정으로 돌아갑니다.")
                    control_num = 0
                    phase_num = 3 

#============================================경로 주행으로 복귀, 로봇 기본 자세로 바꾸고, 수위센서를 확인해야함 ==========================================
        
        elif(phase_num == 10): 
            
            print("phase_num = ", phase_num)
            print("control_num = ", control_num)
            
            control_num += 1
            if(control_num == 1 or control_num == 10):
                send_data("8")       # 기본자세로 복귀

                truss_control = 0   # 화방 개수 변수
                truss_coordinate = 0  # 화방 개수에 따른 위치 조정 변수
                flower_control = 0   # 만개꽃 페이즈 개수 변수
                noflower_count = 0   # 만개꽃 개수 카운트 변수, 2개이상 충족이 안될경우 다른 쪽으로 넘김
                final_truss_num = 0
                truss_num_is_0 = 0
                truss_num_is_1 = 0
                truss_num_is_2 = 0
                final_up_down = 0

            if(control_num == 20):
                send_data("0")    # 이동 명령
              
            if(control_num > 50):    # 분사가 끝난후 어느정도 걷고 나서 페이즈 변경
                phase_num = 0     

        cv2.imshow("Receiving Video", frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

    client_socket.send("o".encode())  #만약 값을 안준다면?, 디버깅용 코드
    time.sleep(0.01) # 0.1초 대기

client_socket.close()
cv2.destroyAllWindows()



0: 480x640 (no detections), 28.6ms
Speed: 17.5ms preprocess, 28.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 18.1ms
Speed: 0.0ms preprocess, 18.1ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)
phase_num =  0

0: 480x640 (no detections), 18.3ms
Speed: 2.0ms preprocess, 18.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 22.4ms
Speed: 0.0ms preprocess, 22.4ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)
phase_num =  0

0: 480x640 (no detections), 20.1ms
Speed: 1.0ms preprocess, 20.1ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 17.7ms
Speed: 1.0ms preprocess, 17.7ms inference, 6.1ms postprocess per image at shape (1, 3, 480, 640)
phase_num =  0

0: 480x640 (no detections), 18.9ms
Speed: 2.0ms preprocess, 18.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detecti